In [1]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

In [2]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(train_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))
validation_dataset = image_dataset_from_directory(validation_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))
test_dataset = image_dataset_from_directory(test_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))

2024-06-18 22:29:12.478057: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 40000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.


In [4]:
import tensorflow as tf

# Função para calcular o tamanho do dataset
def calculate_dataset_size(dataset):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    return dataset_size

# Função para dividir o dataset em três subsets
def split_dataset(dataset, fractions):
    dataset_size = calculate_dataset_size(dataset)
    split_sizes = [int(fraction * dataset_size) for fraction in fractions]
    
    dataset_1 = dataset.take(split_sizes[0])
    dataset_2 = dataset.skip(split_sizes[0]).take(split_sizes[1])
    dataset_3 = dataset.skip(split_sizes[0] + split_sizes[1])
    
    return dataset_1, dataset_2, dataset_3

# Frações do dataset que você quer dividir (devem somar 1)
fractions = [0.4, 0.3, 0.3]

# Dividir o dataset de treinamento em três subsets
train_dataset_1, train_dataset_2, train_dataset_3 = split_dataset(train_dataset, fractions)
validation_dataset_1, validation_dataset_2, validation_dataset_3 = split_dataset(validation_dataset, fractions)

# Calculando os tamanhos dos subsets de treinamento
split_train_size_1 = calculate_dataset_size(train_dataset_1)
split_train_size_2 = calculate_dataset_size(train_dataset_2)
split_train_size_3 = calculate_dataset_size(train_dataset_3)

# Calculando os tamanhos dos subsets de validação
split_validation_size_1 = calculate_dataset_size(validation_dataset_1)
split_validation_size_2 = calculate_dataset_size(validation_dataset_2)
split_validation_size_3 = calculate_dataset_size(validation_dataset_3)

# Mostrando os tamanhos dos subsets de treinamento
print(f"Tamanho do subset 1 do conjunto de treinamento: {split_train_size_1}")
print(f"Tamanho do subset 2 do conjunto de treinamento: {split_train_size_2}")
print(f"Tamanho do subset 3 do conjunto de treinamento: {split_train_size_3}")

# Mostrando os tamanhos dos subsets de validação
print(f"Tamanho do subset 1 do conjunto de validação: {split_validation_size_1}")
print(f"Tamanho do subset 2 do conjunto de validação: {split_validation_size_2}")
print(f"Tamanho do subset 3 do conjunto de validação: {split_validation_size_3}")

Tamanho do subset 1 do conjunto de treinamento: 500
Tamanho do subset 2 do conjunto de treinamento: 375
Tamanho do subset 3 do conjunto de treinamento: 375
Tamanho do subset 1 do conjunto de validação: 125
Tamanho do subset 2 do conjunto de validação: 93
Tamanho do subset 3 do conjunto de validação: 95


In [5]:
from keras.applications.vgg19 import VGG19

conv_base = VGG19(weights="imagenet", include_top=False) 
conv_base.trainable = True

# Deixar todas as camadas, exceto as últimas quatro, não treináveis (congeladas)
for layer in conv_base.layers[:-4]: 
    layer.trainable = False

In [ ]:
from tensorflow import keras

#Adicionar o modelo treinado com feature extraction
base_model = keras.models.load_model('ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5')

In [85]:
from tensorflow import keras
from keras import layers

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg19.preprocess_input(x) 
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x) 
model = keras.Model(inputs, outputs)

In [104]:
#A partir deste bloco iremos treinar o modelo para os sub datasets
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-4), metrics=['accuracy']) 

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

In [105]:
#Subset 2
history = model.fit(train_dataset_2, epochs=2, validation_data=validation_dataset_2, batch_size=64, callbacks=[early_stopping, reduce_lr])

Epoch 1/2


375/375 [==============================] - 1746s 5s/step - loss: 16.5633 - accuracy: 0.6869 - val_loss: 6.7114 - val_accuracy: 0.8377 - lr: 1.0000e-04
Epoch 2/2
375/375 [==============================] - 1747s 5s/step - loss: 12.4871 - accuracy: 0.7064 - val_loss: 5.7143 - val_accuracy: 0.8485 - lr: 1.0000e-04


In [106]:
#Subset 1
history = model.fit(train_dataset_1, epochs=2, validation_data=validation_dataset_1, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

Epoch 1/2
500/500 [==============================] - 2336s 5s/step - loss: 10.9211 - accuracy: 0.7050 - val_loss: 4.8112 - val_accuracy: 0.8465 - lr: 1.0000e-04
Epoch 2/2
500/500 [==============================] - 2322s 5s/step - loss: 9.3581 - accuracy: 0.7101 - val_loss: 3.9973 - val_accuracy: 0.8540 - lr: 1.0000e-04


In [107]:
#model.save('ModelT_transferLearning_fineTuning_WithDataAumentation_OnlyClassification.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
